In [ ]:
import time
import urllib.parse

import autoshop
import pandas as pd
import pygsheets

In [ ]:
driver = autoshop.login()

In [ ]:
time.sleep(1)
driver.get("https://www.tesco.com/groceries/en-GB/orders")

In [ ]:
xpath_make_changes = "//span[text()='Make changes']"
element = autoshop.wait_and_click(driver=driver, value=xpath_make_changes)

In [ ]:
all_food = autoshop.get_all_food()

In [ ]:
URL_TEMPLATE = "https://www.tesco.com/groceries/en-GB/search?query={query}&page={page}&count={count}"

def get_url(
    query: str,
    page: int = 1,
    count: int = 24,
    url_template: str = URL_TEMPLATE,
) -> str:
    return url_template.format(
        query=urllib.parse.quote_plus(query),
        page=page,
        count=count,
    )

In [ ]:
def get_food_elements(driver: autoshop.webdriver.Chrome):
    try:
        elements = autoshop.wait_and_get_all(
            driver=driver,
            value=(
                "//ul[@class = 'product-list grid']"
                "//li"
                "//a[starts-with(@href, '/groceries/en-GB/products/')]"
            ),
        )
    except KeyboardInterrupt as e:
        raise e
    except:
        elements = []

    # Don't want the elements where the text is empty
    elements = [
        element
        for element in elements 
        if element.children()[0].text == ""
    ]

    # Don't want the sponsored items
    elements = [
        element
        for element in elements 
        if not element.find_element(by=autoshop.By.XPATH, value="..").text.startswith("Sponsored")
    ]
    return elements


def get_price(element):
    try:
        return element.find_element(by=autoshop.By.XPATH, value=".//p[contains(text(), '£') and not(contains(text(), '/each')) and @class != 'product-info-message']").text.replace("£", "")
    except KeyboardInterrupt as e:
        raise e
    except:
        return float("nan")

In [ ]:
df_food = pd.DataFrame(all_food["values"][1:], columns=all_food["values"][0])

In [ ]:
elements

In [ ]:
list_df = []

for _, row in df_food.iterrows():
    search = row["search"]
    if search is None or search == "":
        name = row["name"]
    else:
        name = search
    autoshop.logger.info(f"{name=}")
    
    url = get_url(query=name)
    autoshop.logger.info(f"{url=}")
    driver.get(url=url)
    
    elements = get_food_elements(driver=driver)
    
    df_temp = (
        pd.DataFrame(dict(element=elements))
        .assign(
            food=name,
            parent=lambda x: x["element"].apply(lambda e: e.find_element(by=autoshop.By.XPATH, value="..")),
            raw=lambda x: x["parent"].apply(lambda e: e.get_attribute("innerHTML")),
            description=lambda x: x["parent"].apply(lambda e: e.find_element(by=autoshop.By.XPATH, value=".//a[starts-with(@href, '/groceries/en-GB/products/')]//span[text() != '']").text),
            link=lambda x: x["element"].apply(lambda e: e.get_attribute("href")),
            image=lambda x: '=IMAGE("' + x["parent"].apply(lambda e: e.find_element(by=autoshop.By.XPATH, value=".//img").get_attribute("srcset")).str.split(" ").str[0] + '")',
            price=lambda x: pd.to_numeric(x["parent"].apply(get_price), errors="coerce"),
            datetime=pd.Timestamp.now(),
        )
        .drop(columns=["element", "parent"])
    )
    
    if df_temp.empty:
        autoshop.logger.warning(f"No data found for {name=}, {url=}")
    
    list_df.append(df_temp)
    
df = pd.concat(list_df, ignore_index=True)

In [ ]:
google_client = pygsheets.authorize(service_file='food-api-383412-1c354585502d.json')
sheet = google_client.open_by_key(key='1WVGEW5ni7xUs6o5Bs_8joM2iKZclX7hO3lTq2Wp8H-U')
worksheet = sheet.worksheet_by_title("food")

worksheet.set_dataframe(df, start=(1, 1))